# NeuroSymbolic-T4: Complete Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tommaso-R-Marena/NeuroSymbolic-T4/blob/main/notebooks/NeuroSymbolic_T4_Demo.ipynb)

This notebook demonstrates the complete NeuroSymbolic-T4 system on Google Colab with T4 GPU.

## Features Demonstrated:
- Neural perception with concept grounding
- Symbolic reasoning with forward/backward chaining
- Query-based inference with explanations
- Custom rule definition
- Performance benchmarking on T4 GPU

## 1. Setup and Installation

In [ ]:
# Verify T4 GPU
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Clone repository
!git clone https://github.com/Tommaso-R-Marena/NeuroSymbolic-T4.git
%cd NeuroSymbolic-T4

# Install dependencies
!pip install -q -r requirements.txt

print("\n✅ Installation complete!")

## 2. Import and Initialize System

In [ ]:
import torch
import numpy as np
from neurosymbolic import NeurosymbolicSystem
import time

# Initialize system
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = NeurosymbolicSystem(
    perception_config={
        "backbone": "efficientnet_b0",
        "feature_dim": 512,
        "num_concepts": 100,
    }
).to(device)

model.eval()
print("\n✅ Model initialized successfully!")

## 3. Neural Perception Demo

In [ ]:
# Generate sample image
print("Generating sample image...")
image = torch.randn(1, 3, 224, 224).to(device)

# Perception
print("\nRunning neural perception...")
with torch.no_grad():
    perception_output = model.perceive(image, threshold=0.6)

# Display detected concepts
symbolic_scene = perception_output["symbolic"][0]
print(f"\n✅ Detected {len(symbolic_scene)} concepts:")

if symbolic_scene:
    for concept, confidence in sorted(symbolic_scene, key=lambda x: x[1], reverse=True)[:10]:
        print(f"  {concept:20s}: {confidence:.3f}")
else:
    print("  No concepts detected above threshold")

## 4. Symbolic Reasoning Demo

In [ ]:
# Full forward pass (perception + reasoning)
print("Running complete neurosymbolic pipeline...")

with torch.no_grad():
    output = model.forward(image, threshold=0.6)

reasoning = output["reasoning"][0]
print(f"\n✅ Derived {reasoning['num_derived']} new facts through reasoning")

if reasoning["derived_facts"]:
    print("\nDerived facts:")
    for pred, args, conf in reasoning["derived_facts"][:10]:
        print(f"  {pred}{args}: {conf:.3f}")
else:
    print("\nNo new facts derived (try lowering threshold or adding more rules)")

## 5. Query-Based Inference

In [ ]:
# Query: Is there something dangerous?
query = ("dangerous", ("obj0",))

print(f"Query: {query[0]}{query[1]}")

with torch.no_grad():
    proofs = model.query(image, query, threshold=0.5)

print(f"\n✅ Found {len(proofs)} proof(s)")

if proofs:
    print("\nTop proof:")
    proof = proofs[0]
    print(f"Confidence: {proof['confidence']:.3f}")
    print("\nProof steps:")
    for step in proof["proof"]:
        print(f"  - {step}")
else:
    print("\nNo proofs found for this query")

## 6. Explanation Generation

In [ ]:
# Generate explanation for a fact
fact_to_explain = ("vehicle", ("obj0",))

print(f"Explaining: {fact_to_explain[0]}{fact_to_explain[1]}")

with torch.no_grad():
    explanations = model.explain_prediction(image, fact_to_explain, threshold=0.5)

print(f"\n✅ Generated {len(explanations)} explanation(s)")

if explanations:
    print("\n" + explanations[0])
else:
    print("\nNo explanation found (fact may not hold for this input)")

## 7. Custom Rules

In [ ]:
# Add custom domain rules
print("Adding custom rules...")

# Rule 1: Large + Red = Important
model.reasoner.add_rule(
    head=("important", ("?x",)),
    body=[("large", ("?x",)), ("red", ("?x",))],
    confidence=0.9
)
print("  ✓ Rule: important(X) :- large(X) AND red(X) [0.9]")

# Rule 2: Important + Urgent = Priority
model.reasoner.add_rule(
    head=("priority", ("?x",)),
    body=[("important", ("?x",)), ("urgent", ("?x",))],
    confidence=0.95
)
print("  ✓ Rule: priority(X) :- important(X) AND urgent(X) [0.95]")

# Add test facts
model.reasoner.add_fact("large", ("test_obj",), 0.9)
model.reasoner.add_fact("red", ("test_obj",), 0.85)
model.reasoner.add_fact("urgent", ("test_obj",), 0.8)

# Forward chain
num_derived = model.reasoner.forward_chain()
print(f"\n✅ Derived {num_derived} new facts\n")

# Check derived facts
important_conf = model.reasoner.query("important", ("test_obj",))
priority_conf = model.reasoner.query("priority", ("test_obj",))

if important_conf:
    print(f"important(test_obj): {important_conf:.3f}")
if priority_conf:
    print(f"priority(test_obj): {priority_conf:.3f}")

## 8. Performance Benchmarking

In [ ]:
print("Benchmarking inference speed on T4 GPU...")

# Warmup
for _ in range(10):
    x = torch.randn(1, 3, 224, 224).to(device)
    with torch.no_grad():
        _ = model.forward(x)

# Benchmark
torch.cuda.synchronize()
times = []

num_iterations = 100
for i in range(num_iterations):
    x = torch.randn(1, 3, 224, 224).to(device)
    
    start = time.time()
    with torch.no_grad():
        _ = model.forward(x)
    torch.cuda.synchronize()
    end = time.time()
    
    times.append(end - start)
    
    if (i + 1) % 20 == 0:
        print(f"  Progress: {i + 1}/{num_iterations}")

# Statistics
mean_time = np.mean(times) * 1000
std_time = np.std(times) * 1000
fps = 1.0 / np.mean(times)

print(f"\n✅ Benchmark Results (T4 GPU):")
print(f"  Mean latency: {mean_time:.2f} ± {std_time:.2f} ms")
print(f"  Throughput: {fps:.1f} FPS")
print(f"  GPU Memory: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")

## 9. Run All Examples

In [ ]:
# Run provided examples
print("Running all provided examples...\n")

!python examples/basic_usage.py

## 10. Summary

This notebook demonstrated:

✅ Neural perception with EfficientNet backbone
✅ Symbolic reasoning with forward/backward chaining
✅ Query-based inference with proof generation
✅ Explanation generation for predictions
✅ Custom rule definition and integration
✅ Performance benchmarking on T4 GPU

**Expected Performance:**
- Latency: ~20-30ms per image
- Throughput: ~40-50 FPS
- Memory: ~2-3 GB VRAM

**Next Steps:**
- Train on custom dataset
- Add domain-specific rules
- Integrate with real applications
- Explore multi-object scenes

For more information, visit: [GitHub Repository](https://github.com/Tommaso-R-Marena/NeuroSymbolic-T4)